In [1]:
#dependencias
import pandas as pd
import unicodedata
import re
import difflib

#rutas de los archivos
path21 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/raw/Ticket2021.csv'
path22 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/raw/Ticket2022.csv'
path23 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/raw/Ticket2023.csv'
path24 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/raw/Ticket2024.csv'
path25 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/raw/Ticket2025.csv'

#carpetas de guardado
folder_p = "/home/ubuntu/projects/historical-analysis-management-tickets/data/processed"
%store folder_p

Stored 'folder_p' (str)


In [2]:
#dataframes
df2021 = pd.read_csv(path21, sep=';')
df2022 = pd.read_csv(path22, sep=';')
df2023 = pd.read_csv(path23, sep=';')
df2024 = pd.read_csv(path24, sep=';', encoding='latin1')
df2025 = pd.read_csv(path25, sep=';', encoding='latin1')

#diccionario de dataframes
dict_df = {
    2021: df2021,
    2022: df2022,
    2023: df2023,
    2024: df2024,
    2025: df2025
}

In [3]:
# Función para normalizar nombres de columnas
def normalize_col(name):
    if not isinstance(name, str):
        return name
    s = unicodedata.normalize('NFKD', name)
    s = s.encode('ascii', 'ignore').decode('ascii')
    s = s.lower().strip()
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^0-9a-z_]", "", s)
    return s

# Aplicar normalizacion de cols
for year, df in dict_df.items():
    df.columns = [normalize_col(c) for c in df.columns]

# df2021 como referencia
canonical_cols = list(dict_df[2021].columns)
print(f'Columnas bases df2021 ({len(canonical_cols)}): {canonical_cols}\n')

# Alinear otros dfs a las cols de df2021
for year, df in dict_df.items():
    if year == 2021:
        continue
    cols = list(df.columns)
    rename_map = {}
    renamed = []
    for c in cols:
        if c in canonical_cols:
            continue
        matches = difflib.get_close_matches(c, canonical_cols, n=1, cutoff=0.8)
        if matches:
            rename_map[c] = matches[0]
            renamed.append((c, matches[0]))
    if rename_map:
        df.rename(columns=rename_map, inplace=True)
    missing = [c for c in canonical_cols if c not in df.columns]
    extra = [c for c in df.columns if c not in canonical_cols]
    df = df.reindex(columns=canonical_cols)
    dict_df[year] = df
    print(f"Año {year}: renombradas {len(renamed)} columnas; faltan {len(missing)}; columnas extra eliminadas: {len(extra)}")
    if renamed:
        print('  ejemplos de renombrado:', renamed[:10])
    if missing:
        print('  ejemplos de faltantes:', missing[:10])

# Resumen final
cols_by_year = {year: set(df.columns) for year, df in dict_df.items()}
all_cols = set.union(*cols_by_year.values())
common = set.intersection(*cols_by_year.values())
print('\nColumnas comunes (presentes en todos):')
print(sorted(common))
print('\nColumnas totales detectadas:')
print(sorted(all_cols))

Columnas bases df2021 (17): ['id', 'titulo', 'estado', 'ultima_modificacion', 'fecha_de_apertura', 'prioridad', 'solicitante__solicitante', 'asignado_a__grupo_de_tecnicos', 'asignado_a__tecnico', 'categoria', 'origen_de_la_solicitud', 'ubicacion', 'estadisticas__hora_de_resolucion', 'seguimientos__origen_de_la_solicitud', 'fecha_de_cierre', 'fecha_de_solucion', 'unnamed_16']

Año 2022: renombradas 0 columnas; faltan 0; columnas extra eliminadas: 0
Año 2023: renombradas 0 columnas; faltan 1; columnas extra eliminadas: 0
  ejemplos de faltantes: ['unnamed_16']
Año 2024: renombradas 9 columnas; faltan 0; columnas extra eliminadas: 0
  ejemplos de renombrado: [('iid', 'id'), ('tatulo', 'titulo'), ('altima_modificacia3n', 'ultima_modificacion'), ('asignado_a__grupo_de_tacnicos', 'asignado_a__grupo_de_tecnicos'), ('asignado_a__tacnico', 'asignado_a__tecnico'), ('categoraa', 'categoria'), ('ubicacia3n', 'ubicacion'), ('estadasticas__hora_de_resolucia3n', 'estadisticas__hora_de_resolucion'), (

In [4]:
#estructura head de los dataframes
for year, df in dict_df.items():
    print(f"Estructura del DataFrame {year}:")
    display(df.head())
    print("----"*50)
    print()

Estructura del DataFrame 2021:


,id,titulo,estado,ultima_modificacion,fecha_de_apertura,prioridad,solicitante__solicitante,asignado_a__grupo_de_tecnicos,asignado_a__tecnico,categoria,origen_de_la_solicitud,ubicacion,estadisticas__hora_de_resolucion,seguimientos__origen_de_la_solicitud,fecha_de_cierre,fecha_de_solucion,unnamed_16
0,1059,RV: [GLPI #0174457] RE: ABM de Ricardo Bacho (...,Cerrado,26-12-2022 08:18,25-05-2021 16:50,Media,Ricardo Valdes Valenzuela (368),Sistemas e Infraestructura > DBA,Orieta Catalan Ossandon (9),ABM,E-Mail,NaN,575 días 15 horas 53 minutos,NaN,26-12-2022 08:18,NaN,NaN
1,1058,problemas con el correo (1058),Cerrado,13-12-2022 11:22,26-05-2021 10:00,Media,Nicolas Ignacio Alveal Leiva (778),Soporte y Telecomunicaciones TI > Soporte de C...,Carlos Alvarez Fuentes (2710),Soporte de Campo > Revisión de Equipos,E-Mail,NaN,559 días 6 horas 17 minutos,NaN,13-12-2022 11:22,NaN,NaN


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Estructura del DataFrame 2022:


,id,titulo,estado,ultima_modificacion,fecha_de_apertura,prioridad,solicitante__solicitante,asignado_a__grupo_de_tecnicos,asignado_a__tecnico,categoria,origen_de_la_solicitud,ubicacion,estadisticas__hora_de_resolucion,seguimientos__origen_de_la_solicitud,fecha_de_cierre,fecha_de_solucion,unnamed_16
0,679,RE: Configuracion de Perfil - ''Usuarios Eletr...,Cerrado,05-08-2025 15:58,23-11-2022 12:43,Media,Bernardita Espinoza (2769),Soporte y Telecomunicaciones TI > Soporte de C...,Jose Manuel Carrasco Gonzalez (1520),Soporte de Campo > Configuración de Equipos,E-Mail,NaN,20 días 4 horas 59 minutos,NaN,16-12-2022 08:33,NaN,NaN
1,252,RV: Accesos Sharepoint (252),Cerrado,19-12-2024 16:36,02-12-2022 12:31,Media,Nellyda Pia Ferreira Gomez (7),Sistemas e Infraestructura > Técnico TI,NaN,Técnico TI > Accesos Sharepoint,E-Mail,NaN,2 días 21 horas 21 minutos,NaN,13-12-2022 08:46,NaN,NaN
2,494,RE: ABM - VPN - RED Extender cuenta (494),Cerrado,19-12-2024 16:36,18-11-2022 10:47,Media,Nellyda Pia Ferreira Gomez (7),Sistemas e Infraestructura > Técnico TI,NaN,ABM,E-Mail,NaN,17 días 1 horas 30 minutos,Helpdesk,05-12-2022 12:18,NaN,NaN
3,498,RE: ABM - VPN - RED Extender cuenta (498),Cerrado,19-12-2024 16:36,18-11-2022 11:56,Media,NaN,Sistemas e Infraestructura > Técnico TI,NaN,ABM,E-Mail,NaN,17 días 0 horas 20 minutos,Helpdesk,05-12-2022 12:17,NaN,NaN
4,506,RE: ABM - VPN - RED Extender cuenta (506),Cerrado,19-12-2024 16:36,18-11-2022 13:41,Media,NaN,Sistemas e Infraestructura > Técnico TI,NaN,ABM,E-Mail,NaN,16 días 22 horas 37 minutos,Helpdesk,05-12-2022 12:18,NaN,NaN


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Estructura del DataFrame 2023:


...

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Estructura del DataFrame 2024:


,id,titulo,estado,ultima_modificacion,fecha_de_apertura,prioridad,solicitante__solicitante,asignado_a__grupo_de_tecnicos,asignado_a__tecnico,categoria,origen_de_la_solicitud,ubicacion,estadisticas__hora_de_resolucion,seguimientos__origen_de_la_solicitud,fecha_de_cierre,fecha_de_solucion,unnamed_16
0,74345,Ejecucion Proyecto - Requerimiento Corte & Rep...,En curso (asignada),26-01-2026 16:11,17-12-2024 17:45,Media,Diego Mauricio Hernandez Buitrago (2219),Desarrollo de Sistemas y Proyectos TecnolÃ³gicos,Felix Luciano Vidal Catalan (1797),Actividades Proyectos IMA,Helpdesk,NaN,0 segundos,E-Mail,NaN,NaN,NaN
1,73585,Ejecucion Proyecto - Solicita Requerimiento De...,En curso (asignada),23-01-2026 12:29,10-12-2024 09:04,Media,Diego Mauricio Hernandez Buitrago (2219),Desarrollo de Sistemas y Proyectos TecnolÃ³gicos,Jose Manuel Olmos Quinones (1371),Actividades Proyectos IMA,Helpdesk,NaN,0 segundos,NaN,NaN,NaN,NaN
2,74817,Ejecucion Proyecto - Requerimiento alerta al r...,En curso (asignada),20-01-2026 09:35,23-12-2024 17:32,Media,Diego Mauricio Hernandez Buitrago (2219),Desarrollo de Sistemas y Proyectos TecnolÃ³gicos,Enrique Faundez Cancino (2050),Actividades Proyectos IMA,Helpdesk,NaN,0 segundos,E-Mail,NaN,NaN,NaN
3,55132,Urgente: Equipo JFCINGENIERIA01 desconectado (...,Cerrado,19-01-2026 13:02,24-05-2024 14:49,Muy alta,jfcollao@jfc-ingenieria.com,Soporte y Telecomunicaciones TI > Soporte de C...,Catalina Andrea Rios Sepulveda (3689),Soporte de Campo > RevisiÃ³n de Equipos,E-Mail,NaN,10 minutos,Helpdesk,27-05-2024 08:29,27-05-2024 11:49,NaN
4,51355,ModificaciÃ³n cierre de Reclamos sobre 30 dÃ­a...,Cerrado,11-01-2026 11:12,17-04-2024 18:00,Media,Martina Paz Ferreiro Hormazabal (1024),Resolutores NO TI > Soporte Comercial OSF,Pablo Alvarado Tapia (680),Aplicaciones > OSF,E-Mail,NaN,12 dÃ­as 23 horas 17 minutos,Helpdesk<br>E-Mail,02-05-2024 22:20,NaN,NaN


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Estructura del DataFrame 2025:


,id,titulo,estado,ultima_modificacion,fecha_de_apertura,prioridad,solicitante__solicitante,asignado_a__grupo_de_tecnicos,asignado_a__tecnico,categoria,origen_de_la_solicitud,ubicacion,estadisticas__hora_de_resolucion,seguimientos__origen_de_la_solicitud,fecha_de_cierre,fecha_de_solucion,unnamed_16
0,112632,GestiÃ³n Clave usuario lgarcia / Luis Rodrigo ...,Cerrado,22-01-2026 11:29,22-01-2026 11:29,Media,David Isaac Rodriguez Troncoso (3818),Soporte y Telecomunicaciones TI > Agentes N1,David Isaac Rodriguez Troncoso (3818),MDA > Desbloqueo Cuenta Red,Helpdesk,NaN,11 segundos,NaN,22-01-2026 11:29,23-01-2026 09:29,NaN
1,112631,GestiÃ³n Clave usuario lgarcia / Luis Rodrigo ...,Cerrado,22-01-2026 11:28,22-01-2026 11:28,Media,David Isaac Rodriguez Troncoso (3818),Soporte y Telecomunicaciones TI > Agentes N1,David Isaac Rodriguez Troncoso (3818),MDA > Cambio contraseÃ±a RED,Helpdesk,NaN,10 segundos,NaN,22-01-2026 11:28,23-01-2026 09:28,NaN
2,112620,[SQL] URGENTE FACTURACION HOY Script 4 -(CHILQ...,Cerrado,22-01-2026 11:17,22-01-2026 10:57,Muy alta,Eduardo Francisco Zamora Gonzalez (3971),Sistemas e Infraestructura > DBA,Diego Mauricio Hernandez Buitrago (2219),DBA > Query BD,E-Mail,NaN,19 minutos,NaN,22-01-2026 11:17,22-01-2026 14:57,NaN
3,112615,consulta ticket (112615),Cerrado,22-01-2026 10:48,22-01-2026 10:46,Media,Carlos Joaquin Alberto Escobar Soto (3695),Soporte y Telecomunicaciones TI > Agentes N1,Barbara Aracely Oyarzun Gonzalez (3907),MDA > Consulta Ticket,Helpdesk,NaN,53 segundos,NaN,22-01-2026 10:48,23-01-2026 08:46,NaN
4,112570,RevisiÃ³n de salas Baron 2201 (112570),Cerrado,22-01-2026 10:38,22-01-2026 08:25,Media,Pedro Pablo Zepeda Pozo (537),Soporte y Telecomunicaciones TI > Soporte de C...,Jose Manuel Carrasco Gonzalez (1520),Soporte de Campo > RevisiÃ³n de PerifÃ©ricos >...,Helpdesk,Baron,2 horas 13 minutos,NaN,22-01-2026 10:38,22-01-2026 16:25,NaN


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [5]:
#estructura de los formatos
for year, df in dict_df.items():
    print(f"Formato de las columnas del DataFrame {year}:")
    display(df.info())
    print("----"*50)
    print("----"*50)
    print()

Formato de las columnas del DataFrame 2021:
<class 'pandas.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    2 non-null      int64  
 1   titulo                                2 non-null      str    
 2   estado                                2 non-null      str    
 3   ultima_modificacion                   2 non-null      str    
 4   fecha_de_apertura                     2 non-null      str    
 5   prioridad                             2 non-null      str    
 6   solicitante__solicitante              2 non-null      str    
 7   asignado_a__grupo_de_tecnicos         2 non-null      str    
 8   asignado_a__tecnico                   2 non-null      str    
 9   categoria                             2 non-null      str    
 10  origen_de_la_solicitud                2 non-null      str

None

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Formato de las columnas del DataFrame 2022:
<class 'pandas.DataFrame'>
RangeIndex: 3827 entries, 0 to 3826
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    3827 non-null   int64  
 1   titulo                                3827 non-null   str    
 2   estado                                3827 non-null   str    
 3   ultima_modificacion                   3827 non-null   str    
 4   fecha_de_apertura                     3827 non-nul

None

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Formato de las columnas del DataFrame 2023:
<class 'pandas.DataFrame'>
RangeIndex: 36259 entries, 0 to 36258
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    36259 non-null  int64  
 1   titulo                                36259 non-null  str    
 2   estado                                36259 non-null  str    
 3   ultima_modificacion                   36259 non-null  str    
 4   fecha_de_apertura                     36259 non-

...

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Formato de las columnas del DataFrame 2024:
<class 'pandas.DataFrame'>
RangeIndex: 35004 entries, 0 to 35003
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    35004 non-null  int64  
 1   titulo                                35004 non-null  str    
 2   estado                                35004 non-null  str    
 3   ultima_modificacion                   35004 non-null  str    
 4   fecha_de_apertura                     35004 non-

None

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Formato de las columnas del DataFrame 2025:
<class 'pandas.DataFrame'>
RangeIndex: 36527 entries, 0 to 36526
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    36527 non-null  int64  
 1   titulo                                36527 non-null  str    
 2   estado                                36527 non-null  str    
 3   ultima_modificacion                   36527 non-null  str    
 4   fecha_de_apertura                     36527 non-

None

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [6]:
#nulos
for year, df in dict_df.items():
    print(f"% nulos en el DataFrame {year}:")
    print(f"TOTAL DATOS = {df.shape[0]}")
    display(round(df.isnull().sum()/df.shape[0] * 100))
    print("Duplicados:")
    display(df.duplicated().sum())
    print("----"*50)
    print()

% nulos en el DataFrame 2021:
TOTAL DATOS = 2


id                                        0.0
titulo                                    0.0
estado                                    0.0
ultima_modificacion                       0.0
fecha_de_apertura                         0.0
prioridad                                 0.0
solicitante__solicitante                  0.0
asignado_a__grupo_de_tecnicos             0.0
asignado_a__tecnico                       0.0
categoria                                 0.0
origen_de_la_solicitud                    0.0
ubicacion                               100.0
estadisticas__hora_de_resolucion          0.0
seguimientos__origen_de_la_solicitud    100.0
fecha_de_cierre                           0.0
fecha_de_solucion                       100.0
unnamed_16                              100.0
dtype: float64

Duplicados:


np.int64(0)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

% nulos en el DataFrame 2022:
TOTAL DATOS = 3827


...

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

% nulos en el DataFrame 2025:
TOTAL DATOS = 36527


id                                        0.0
titulo                                    0.0
estado                                    0.0
ultima_modificacion                       0.0
fecha_de_apertura                         0.0
prioridad                                 0.0
solicitante__solicitante                  0.0
asignado_a__grupo_de_tecnicos             3.0
asignado_a__tecnico                       4.0
categoria                                 6.0
origen_de_la_solicitud                    0.0
ubicacion                                94.0
estadisticas__hora_de_resolucion          0.0
seguimientos__origen_de_la_solicitud     72.0
fecha_de_cierre                           0.0
fecha_de_solucion                        31.0
unnamed_16                              100.0
dtype: float64

Duplicados:


np.int64(0)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [7]:
# Guardar los DataFrames normalizados
for year, df in dict_df.items():
    name = f"df{year}.csv"
    df.to_csv(f"{folder_p}/{name}")